In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
import dash_table
from dash.dependencies import Input, Output
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import CRUDModule




from CRUDModule import AnimalShelter



###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "renez"
database = "AAC"
shelter = AnimalShelter(username, password, database)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.A([
        html.Img(
    src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width':'100px'})],
        href = 'www.snhu.edu'),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dcc.Dropdown(
        id='filter-dropdown',
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
            {'label': 'Reset Filter', 'value': 'RF'}
        ],
        value = '',
        placeholder="Select the Type of Rescue Dog is Needed"
    ),
    dash_table.DataTable(
        id ='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False, 
        filter_action = "native",
        sort_action = "native", 
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False, 
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,

    ),
    html.H3("Enrique Zarate SNHU - Dashboard Identifier"),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    html.Div(
            id='barGraph',
            className='col s12 m6',
            ),
    html.H3("Enrique Zarate SNHU - Dashboard Identifier"),
    html.A([
        html.Img(
    src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width':'100px'})],
        href = 'www.snhu.edu')
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Uses the dropdown menu to filter data.
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-dropdown', 'value')]
)
def update_dt(value):   
    if value == '':
        df = pd.DataFrame.from_records(shelter.read({}))
    elif value == 'WR':
        df = pd.DataFrame(shelter.read({ "$and" : [{ "breed" : {"$in" : ['Labrador Retriever Mix',
                                                             'Chesapeake Bay Retriever',
                                                              'Newfoundland']}},
                                      {"sex_upon_outcome" : "Intact Female"},
                                                 {"age_upon_outcome_in_weeks" : {'$gte':26, '$lte':156}}
                                                  ]}))
    elif value == 'MWR':
        df = pd.DataFrame(shelter.read({ "$and" : [{ "breed" : {"$in" : ['German Shepherd',
                                                             'Alaskan Malamute',
                                                              'Old English Sheepdog',
                                                              'Siberian Huskey',
                                                              'Rottweiler']}},
                                        {"sex_upon_outcome" : "Intact Male"},
                                        {"age_upon_outcome_in_weeks" : {'$gte':26, '$lte':156}}
                                                  ]}))
    elif value == 'DIT':
        df = pd.DataFrame(shelter.read({ "$and" : [{ "breed" : {"$in" : ['Doberman Pinscher',
                                                             'German Shepherd',
                                                              'Golden Retriever',
                                                              'Bloodhound',
                                                              'Rottweiler']}},
                                        {"sex_upon_outcome" : "Intact Male"},
                                        {"age_upon_outcome_in_weeks" : {'$gte':20, '$lte':300}}
                                                  ]}))
    elif value == 'RF':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    data = df.to_dict('records')
                      
    return data

@app.callback(
    Output('barGraph', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
    Input('datatable-id', 'derived_virtual_selected_rows')]
)   
def updateGraph(rows,derived_virtual_selected_rows):
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []
        
    dff = df if rows is None else pd.DataFrame(rows)
    
    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]
    
    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [
                    {
                        "x": dff["breed"],
                        "y": dff.groupby("breed").size(),
                        "type": "bar",
                        "marker": {"color": colors},
                    }
                ],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {
                        "automargin": True,
                        "title": {"text": column}
                    },
                    "height": 250,
                    "margin": {"t": 10, "l": 10, "r": 10},
                },
            },
        )
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        for column in ["breed"] if column in dff
    ]
    
    

#This callback grabs the location of the first animal in the table and places it on geolocation map.
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '750px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]



app